In [ ]:
################################################################################################
# Create graph data for a given project
################################################################################################
import pandas as pd

repo_name = "piggymetrics"
option_data = pd.read_csv(f'../data/options/{repo_name}_options_internal.csv')
artifacts_from_options = option_data['File Path'].unique()

# Create a list of unique nodes
concepts = option_data['Concept'].unique()
artifacts = option_data['File Path'].unique()
options = option_data[['File Path', 'Option', 'Values', 'Internal Changes']].apply(tuple, axis=1)

# Add nodes
nodes = [{'id': concept, 'type': 'concept'} for concept in concepts]
nodes += [{'id': artifact, 'type': 'artifact'} for artifact in artifacts]
nodes += [{'id': f"{option[0]}:{option[1]}", 'values': str(option[2]), 'changed_internally': option[3], 'type': 'option', } for option in options]

# Define links
links = []

# Link concepts to artifacts
for _, row in option_data.iterrows():
    links.append({'source': row['Concept'], 'target': row['File Path'], 'type': 'concept-artifact'})
    
# Link artifacts to options
for _, row in option_data.iterrows():
    artifact = row["File Path"]
    option = row["Option"]
    links.append({'source': row['File Path'], 'target': f"{artifact}:{option}", 'type': 'artifact-option'})


################################################################################################
# Add additional links to the graph data (Co-Changes of config options)
################################################################################################
# Links options to options
option_relations_path = f'../data/cochanges/{repo_name}_cochanges.csv'
option_relations = pd.read_csv(option_relations_path)

for _, row in option_relations.iterrows():
    # Create a link for the concept relationship
    links.append({
        'source': f"{row['Artifact1']}:{row['Option1']}",
        'target': f"{row['Artifact2']}:{row['Option2']}",
        'source_option': row['Option1'].split(":")[-1],
        'target_option': row['Option2'].split(":")[-1],
        'type': 'option-option',
        'commit_window': row['Commit Window'],
        'internal_weight': float(row['Percentage Internally']),
        'internal_count': row['Changed Internally'], 
    })


# Output the data
graph_data = {'nodes': nodes, 'links': links}

# Save the graph data as JSON for D3.js
import json
output_path = f'../data/graph_data/{repo_name}_graph_data.json'
with open(output_path, 'w') as f:
    json.dump(graph_data, f)